In [1]:
import os
import glob
#import tqdm
import numpy as np
#import pandas as pd
import multiprocessing
import pydicom as dicom
import nibabel as nib
#from keras import utils as kutils
from matplotlib import pyplot as plt

def load_image(label_fpath, transpose=False):
    #encode_name = label_fpath[-39: -7]
    label_data = nib.load(label_fpath)
    label_array = label_data.get_fdata()
    if transpose:
        label_array = np.transpose(label_array, axes=(2, 1, 0))
    return  label_array#, encode_name

In [2]:
ls

Untitled.ipynb                   masking_dilation_boxTumor(newStart).ipynb
basic_info1(boxes_and_xy).ipynb  tumor_masking_dilation(newStart).ipynb


In [3]:
cd ..

/home/u/wartytw


In [4]:
import pandas as pd
info = pd.read_csv('/data2/pancreas/Nifti_data/data_list.csv')

In [5]:
info['total'] = [1]*len(info)
pd.DataFrame(info.groupby(['resection']).sum().total)

,total
resection,
0.0,80
1.0,61


In [11]:
pd.DataFrame(info.groupby(['resection', 'SMV']).sum().total)




total
resection SMV       
1.0       0       32
          1        6
          x       23

In [15]:
pd.DataFrame(info.groupby(['resection', 'SMV', 'CBD',  'retroperitoneal']).sum().total)



total
resection SMV CBD retroperitoneal       
1.0       0   0   0                   14
                  1                   14
              1   0                    4
          1   0   0                    5
                  1                    1
          x   x   x                   23

In [11]:
pd.DataFrame(info.groupby(['resection', 'CBD']).sum().total)

total
resection CBD       
1.0       0       34
          1        4
          x       23

In [12]:
pd.DataFrame(info.groupby(['resection', 'uncinate']).sum().total)

total
resection uncinate       
1.0       0            19
          1            18
          1?            1
          x            23

In [14]:
pd.DataFrame(info.groupby(['resection', 'retroperitoneal']).sum().total)

total
resection retroperitoneal       
1.0       0                   23
          1                   15
          x                   23

In [32]:
image_path = "/data2/pancreas/Nifti_data/image"
label_path = "/data2/pancreas/Nifti_data/label"


resec_train = np.load("resec_train.npy")
resec_val = np.load("resec_val.npy")
y_train = np.load("y_train.npy")
y_val = np.load("y_val.npy")

shape_tumor = np.load("box_tumor.npy")
resec = np.concatenate([resec_train, resec_val], axis = 0)
y = np.concatenate([y_train, y_val], axis = 0)

In [66]:
info.iloc[np.where(info['case_id'] == "PC673")]['SMV'].values[0]


'x'

In [70]:

position_type = ['SMV', 'CBD', 'uncinate', 'retroperitoneal']
resec_with_pos = []
y_with_pos = [[] for i in range(4)]
for idx, name in enumerate(resec):
    for i, t in enumerate(position_type):
        print(name)
        #print(type(position_type[i]))
        print(info.iloc[np.where(info['case_id'] == name)][position_type[i]].values[0])
        if info.iloc[np.where(info['case_id'] == name)][position_type[i]].values[0] == '0' or info.iloc[np.where(info['case_id'] == name)][position_type[i]].values[0] == '1':
            resec_with_pos.append(name)
            y_with_pos[i].append(y[idx])
            continue

PC673
x
PC673
x
PC673
x
PC673
x
PC568
nan
PC568
nan
PC568
nan
PC568
nan
PC303
nan
PC303
nan
PC303
nan
PC303
nan
PC477
1
PC477
0
PC477
0
PC477
1
PC554
nan
PC554
nan
PC554
nan
PC554
nan
PC911
nan
PC911
nan
PC911
nan
PC911
nan
PC898
0
PC898
0
PC898
0
PC898
1
PC582
nan
PC582
nan
PC582
nan
PC582
nan
PC579
nan
PC579
nan
PC579
nan
PC579
nan
PC773
x
PC773
x
PC773
x
PC773
x
PC559
nan
PC559
nan
PC559
nan
PC559
nan
PC545
1
PC545
0
PC545
0
PC545
0
PC572
nan
PC572
nan
PC572
nan
PC572
nan
PC551
nan
PC551
nan
PC551
nan
PC551
nan
PC769
0
PC769
0
PC769
0
PC769
1
PC648
nan
PC648
nan
PC648
nan
PC648
nan
PC304
x
PC304
x
PC304
x
PC304
x
PC834
nan
PC834
nan
PC834
nan
PC834
nan
PC580
0
PC580
0
PC580
1
PC580
0
PC872
nan
PC872
nan
PC872
nan
PC872
nan
PC104
0
PC104
0
PC104
1
PC104
1
PC110
x
PC110
x
PC110
x
PC110
x
PC844
x
PC844
x
PC844
x
PC844
x
PC456
1
PC456
0
PC456
0
PC456
0
PC915
nan
PC915
nan
PC915
nan
PC915
nan
PC686
nan
PC686
nan
PC686
nan
PC686
nan
PT8
1
PT8
0
PT8
0
PT8
0
PC576
nan
PC576
nan
PC576
nan
PC

In [73]:
print(len(np.unique(resec_with_pos)))

36


# Load existing data

In [16]:
image_path = "/data2/pancreas/Nifti_data/image"
label_path = "/data2/pancreas/Nifti_data/label"

In [17]:
resec_train = np.load("resec_train.npy")
resec_val = np.load("resec_val.npy")
y_train = np.load("y_train.npy")
y_val = np.load("y_val.npy")

In [18]:
resec = np.concatenate([resec_train,resec_val], axis = 0)
y = np.concatenate([y_train, y_val], axis = 0)

In [19]:
box_tumor = np.load("box_tumor.npy")
box_pancreas = np.load("box_pancreas.npy")

# Count the numbers of 2D input for each type

In [52]:
tt_name = pd.DataFrame(info.groupby(['resection', 'retroperitoneal']).case_id)[1][1].iloc[0]

In [60]:
np.where(resec==tt_name)[0][0]

89

In [61]:
print(resec[89])
print(tt_name)

PC42
PC42


In [63]:
box_tumor[89, 5] - box_tumor[89, 4]

5.0

In [65]:
#pd.DataFrame(info.groupby(['resection', pos_type[0]]).case_id)[1][1].iloc[0]
pos_type = ['SMV', 'CBD',  'retroperitoneal']
pos_01x_num = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
pos_01x = [[[], [], []], [[], [], []], [[], [], []]]
for idx, t in enumerate(pos_type):
    for i in range(3):
        N = len(pd.DataFrame(info.groupby(['resection', t]).case_id)[1][i])
        for n in range(N):
            name = pd.DataFrame(info.groupby(['resection', t]).case_id)[1][i].iloc[n]
            name_idx = np.where(resec==name)[0][0]
            val = box_tumor[name_idx, 5] - box_tumor[name_idx, 4]
            pos_01x_num[idx][i]+=val

In [66]:
pos_01x_num

[[101.0, 44.0, 79.0], [126.0, 19.0, 79.0], [102.0, 43.0, 79.0]]

In [40]:
pos_type = ['SMV', 'CBD',  'retroperitoneal']

for idx, t in enumerate(pos_type):
    pd.DataFrame(info.groupby(['resection', t]).case_id)[1][]
    

0
1
2


In [34]:
pd.DataFrame(info.groupby(['resection', 'SMV']).sum().total)

total
resection SMV       
1.0       0       32
          1        6
          x       23